In [ ]:
import torch
from models.BiSpecNet import BinSpecCNN
import cv2 as cv
import numpy as np
import torchaudio
import torchvision.transforms as transforms
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

from torch.optim import Adam

In [ ]:
def wavToSpec(wavname):
	waveform, sample_rate = torchaudio.load(
		filepath=wavname, frame_offset=10000, num_frames=16000)
	if len(waveform[0]) == 0:
		print("ERROR")
		return

	specgram = torchaudio.transforms.Spectrogram()(waveform)
	if specgram.size(2) != 81:
		print("WARN")
		return
	image_arr = specgram.log2()[0, :, :].numpy()
	filepath = '../cache/1.png'
	plt.imsave(fname=filepath, arr=image_arr, format='png', cmap='gray')


def read_img():
	image = '../cache/1.png'
	pic = cv.imread(image, cv.IMREAD_GRAYSCALE)
	dataItem = cv.resize(pic, (201, 201), interpolation=cv.INTER_CUBIC)
	transf = transforms.ToTensor()
	imageTensor = transf(dataItem)
	data = imageTensor.reshape((1, 1, 201, 201))
	return data


In [ ]:
def process(filename):
	wavToSpec(filename)
	data = read_img()

	model = BinSpecCNN(classCount=10)
	model = torch.load('../model/bin,97.54,200,10.pt', map_location='cpu')
	out = model(data)
	_, predicted = torch.max(out.data, dim=1)
	return predicted.item() + 2

In [ ]:
import tkinter as tk
from tkinter import *
import tkinter.filedialog
import cv2

name = 1


def choose_file():
	selectFileName = tk.filedialog.askopenfilename(title='Choose File')
	print(process(selectFileName))
	name = 1


def window():
	root = tk.Tk()
	root.geometry('650x450+150+100')
	root.title('test')
	root.resizable(False, False)

	global e
	e = tk.StringVar()
	e_entry = tk.Entry(root, width=68, textvariable=e)
	e_entry.pack()

	sumbit_btn = tk.Button(root, text="Choose File", bg='yellow', command=choose_file)
	sumbit_btn.pack()

	root.mainloop()


window()